# 序列化
本文档介绍如何将链序列化到磁盘并从磁盘中读取序列化的链。我们使用的序列化格式是json或yaml。当前，只有一些链支持此类型的序列化。我们将随着时间的推移扩大支持的链的数量。


## 将链保存到磁盘
首先，让我们讨论如何将链保存到磁盘。这可以通过使用`.save`方法并指定带有json或yaml扩展名的文件路径来完成。

In [1]:
from langchain import PromptTemplate, OpenAI, LLMChain
template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=OpenAI(temperature=0), verbose=True)


In [2]:
llm_chain.save("llm_chain.json")

现在，让我们来看看保存文件中的内容。

In [3]:
!cat llm_chain.json

{
    "memory": null,
    "verbose": true,
    "prompt": {
        "input_variables": [
            "question"
        ],
        "output_parser": null,
        "template": "Question: {question}\n\nAnswer: Let's think step by step.",
        "template_format": "f-string"
    },
    "llm": {
        "model_name": "text-davinci-003",
        "temperature": 0.0,
        "max_tokens": 256,
        "top_p": 1,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "n": 1,
        "best_of": 1,
        "request_timeout": null,
        "logit_bias": {},
        "_type": "openai"
    },
    "output_key": "text",
    "_type": "llm_chain"
}

## 从磁盘加载链
我们可以使用`load_chain`方法从磁盘加载链。

In [4]:
from langchain.chains import load_chain

In [5]:
chain = load_chain("llm_chain.json")

In [6]:
chain.run("whats 2 + 2")



> Entering new LLMChain chain...
Prompt after formatting:
Question: whats 2 + 2

Answer: Let's think step by step.

> Finished chain.


' 2 + 2 = 4'

## 分别保存组件
在上面的示例中，我们可以看到提示符和llm配置信息保存在与整个链相同的json中。或者，我们可以将它们分开保存。这经常有用，可以使保存的组件更加模块化。为了做到这一点，我们只需要指定`llm_path`而不是`llm`组件，以及`prompt_path`而不是`prompt`组件。

In [7]:
llm_chain.prompt.save("prompt.json")

In [8]:
!cat prompt.json

{
    "input_variables": [
        "question"
    ],
    "output_parser": null,
    "template": "Question: {question}\n\nAnswer: Let's think step by step.",
    "template_format": "f-string"
}

In [9]:
llm_chain.llm.save("llm.json")

In [10]:
!cat llm.json

{
    "model_name": "text-davinci-003",
    "temperature": 0.0,
    "max_tokens": 256,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0,
    "n": 1,
    "best_of": 1,
    "request_timeout": null,
    "logit_bias": {},
    "_type": "openai"
}

In [11]:
config = {
    "memory": None,
    "verbose": True,
    "prompt_path": "prompt.json",
    "llm_path": "llm.json",
    "output_key": "text",
    "_type": "llm_chain"
}
import json
with open("llm_chain_separate.json", "w") as f:
    json.dump(config, f, indent=2)

In [12]:
!cat llm_chain_separate.json

{
  "memory": null,
  "verbose": true,
  "prompt_path": "prompt.json",
  "llm_path": "llm.json",
  "output_key": "text",
  "_type": "llm_chain"
}

然后，我们可以以同样的方式加载它。

In [13]:
chain = load_chain("llm_chain_separate.json")

In [15]:
chain.run("whats 2 + 2")



> Entering new LLMChain chain...
Prompt after formatting:
Question: whats 2 + 2

Answer: Let's think step by step.

> Finished chain.


' 2 + 2 = 4'